## Overview

This is just another simple snippet to upload a custom container to Vertex AI for prediction, is based in Flask web-server but any kind of web-server works.

### Set the Variables and Create your Work Directory

Tree should be like this:

    .
    ├── ..
    ├── custom_4                    # Main folder for prediction
    │   ├── Dockerfile              # Declarative file to build the container
    │   ├── prediction              # Workdir or folder for code
    │       ├── app.py
    |       ├── requirements.txt    # Code for web serving handler and predictions
    └── ...

In [8]:
IMAGE_URI = 'us-central1-docker.pkg.dev/jchavezar-demo/trainings/flask_custom:latest'

In [3]:
!rm -fr custom_4
!mkdir custom_4
!mkdir custom_4/prediction

### Create the Config Files and Prediction Code

In [24]:
%%writefile custom_4/prediction/app.py

import os
import sys
from flask import Flask, request, Response, jsonify

app = Flask(__name__)

@app.route(os.environ['AIP_HEALTH_ROUTE'])
def health():
    status_code =  Response(status=200)
    print('[INFO] ------ Testing', file=sys.stderr)
    return status_code

@app.route(os.environ['AIP_PREDICT_ROUTE'], methods=['GET', 'POST'])
def prediction():
    print('[INFO] ------ Prediction Entry', file=sys.stderr)
    return jsonify({"text": "text"})

Overwriting custom_4/prediction/app.py


In [21]:
%%writefile custom_4/prediction/requirements.txt

flask

Overwriting custom_4/prediction/requirements.txt


In [22]:
%%writefile custom_4/Dockerfile

FROM python:3.7-buster

RUN mkdir my-model

WORKDIR /prediction

COPY prediction/ /prediction
RUN pip install -r requirements.txt 

# Flask Env Variable
ENV FLASK_APP=app

# Expose port 8080
EXPOSE 8080

CMD flask run --host=0.0.0.0 --port=8080

Overwriting custom_4/Dockerfile


### Wrap the files and create the container

In [25]:
!docker build -t $IMAGE_URI custom_4/.

Sending build context to Docker daemon  4.608kB
Step 1/8 : FROM python:3.7-buster
 ---> 1902e2432d77
Step 2/8 : RUN mkdir my-model
 ---> Using cache
 ---> cf0209d8f52f
Step 3/8 : WORKDIR /prediction
 ---> Using cache
 ---> c696675a7c00
Step 4/8 : COPY prediction/ /prediction
 ---> 9949e09b7f81
Step 5/8 : RUN pip install -r requirements.txt
 ---> Running in 460b2ef2b63e
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.4/232.4 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 15.6 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Removing intermediate container 460b2ef2b63e
 ---> d96ffd2dc746
Step 6/8 : ENV FLASK_APP=app
 ---> Running in f10a1d769af4
Removing intermediate container f10a1d769af4
 ---> 0b18a702c2a6
Step 7/8 : EXPOSE

### Push the container

In [26]:
!docker push $IMAGE_URI

The push refers to repository [us-central1-docker.pkg.dev/jchavezar-demo/trainings/flask_custom]

57959340: Preparing 
457018bf: Preparing 
fbf9910f: Preparing 
6b36f95d: Preparing 
a8577818: Preparing 
ae58a42a: Preparing 
ed3cf4db: Preparing 
86d6d73d: Preparing 
ee259bc5: Preparing 
fe198f83: Preparing 
c4b93150: Preparing 
3d4fb826: Preparing 
ee259bc5: Pushed     521MB/510.3MBlatest: digest: sha256:7ab78640ad321a795f5f75835113aad3ab18b6ac4485cd09fa2228436b037bf0 size: 3048


### Import the Container into Vertex Model Registry

In [27]:
from google.cloud import aiplatform as aip

model = aip.Model.upload(
    display_name='flask-dummy',
    serving_container_image_uri=IMAGE_URI,
    serving_container_predict_route='/predict',
    serving_container_health_route='/health',
)

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 2.3.0- is an invalid version and will not be supported in a future release
  warnings.warn(


Creating Model
Create Model backing LRO: projects/569083142710/locations/us-central1/models/3074599549122117632/operations/640654982776881152
Model created. Resource name: projects/569083142710/locations/us-central1/models/3074599549122117632
To use this Model in another session:
model = aiplatform.Model('projects/569083142710/locations/us-central1/models/3074599549122117632')


### Create the endpoint where the model will be deployed

In [28]:
endpoint = aip.Endpoint.create(
    display_name='flask-dummy-test'
)

Creating Endpoint
Create Endpoint backing LRO: projects/569083142710/locations/us-central1/endpoints/3215772444082044928/operations/3088361380252745728
Endpoint created. Resource name: projects/569083142710/locations/us-central1/endpoints/3215772444082044928
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/569083142710/locations/us-central1/endpoints/3215772444082044928')


### Deploy the model

In [31]:
endpoint.deploy(
    model=model,
    deployed_model_display_name='flask-dummy-test',
    min_replica_count=1,
    machine_type='n1-standard-2'
)

Deploying Model projects/569083142710/locations/us-central1/models/3074599549122117632 to Endpoint : projects/569083142710/locations/us-central1/endpoints/3215772444082044928
Deploy Endpoint model backing LRO: projects/569083142710/locations/us-central1/endpoints/3215772444082044928/operations/1304935927814029312
Endpoint model deployed. Resource name: projects/569083142710/locations/us-central1/endpoints/3215772444082044928


In [32]:
ENDPOINT_ID="3215772444082044928"
PROJECT_ID="jchavezar-demo"
INPUT_DATA_FILE="INPUT-JSON"

In [40]:
%%bash
curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-central1/endpoints/${ENDPOINT_ID}:predict \
-d '{"test": "test"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    16    0     0  100    16      0     47 --:--:-- --:--:-- --:--:--    47
